TextRNN 利用 RNN（循环神经网络）进行**文本特征抽取**，由于文本本身是一种序列，而 **LSTM 天然适合序列数据的建模**。

TextRNN将句子中每个词的词向量依次输入到双向双层LSTM，分别**将两个方向最后一个有效位置的隐藏层**拼接成一个向量作为文本的表示。

In [1]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(levelname)s: %(message)s')

import random
import numpy as np
import pandas as pd

import torch
# RuntimeError: cuDNN error: CUDNN_STATUS_EXECUTION_FAILED 
torch.backends.cudnn.enabled = False


# set seed
seed = 666
random.seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)

# set cuda
gpu = 0
# 右边是个bool判断，返回True或者False
use_cuda = gpu >= 0 and torch.cuda.is_available()
if use_cuda:
    torch.cuda.set_device(gpu)
    device = torch.device("cuda", gpu)
else:
    device = torch.device("cpu")
logging.info("Use cuda: %s, gpu id: %d.", use_cuda, gpu)

2020-08-05 18:07:23,462 INFO: Use cuda: True, gpu id: 0.


#### load data

In [2]:
%%time
n_fold = 10
n_example = 10000
data_file = './train_set.csv.zip'
train = pd.read_csv(data_file, sep='\t', nrows=n_example)

# 数据直接全局变量
train_texts = train['text'].tolist()
train_labels = train['label'].tolist()
n_total = len(train_labels)

Wall time: 537 ms


In [3]:
def all_data2index(fold_num):
    """读取df，将series转换为list进行处理"""
        
    # 1.所有数据打乱，通过打乱列表的索引来实现 / 通过sklearn的shuffle模块？
    index = list(range(n_total))
    np.random.shuffle(index)
    all_texts = []
    all_labels = []
    for i in index:
        all_texts.append(train_texts[i])
        all_labels.append(train_labels[i])
    
    # 2.将所有数据按照类别进行划分，通过索引实现：字典检查某个键是否存在，不存在，就创建列表，存在则往列表里添加
    label2id = {}
    for i in range(n_total):
        label = str(all_labels[i])
        # 字典检查某个键是否存在？？ 不加.keys?
        if label not in label2id:
            label2id[label] = [i]
        else:
            label2id[label].append(i)
    
    # 3.
    all_index = [[] for _ in range(fold_num)]
    for label, data in label2id.items():
        # print(label, len(data))
        batch_size = int(len(data) / fold_num)
        other = len(data) - batch_size * fold_num
        
        # 对每个类别都进行10折划分
        for i in range(fold_num):
            # if判断用于赋值
            
            # ？
            cur_batch_size = batch_size + 1 if i < other else batch_size
            # print(cur_batch_size)
            batch_data = [data[i * batch_size + b] for b in range(cur_batch_size)]
            
            # 总共包含10个列表，每个列表都包含所有类别的数据
            all_index[i].extend(batch_data)
            # 等价于 all_index = [], all_index.append(batch_data)
            
            
    return all_texts, all_labels, all_index

In [4]:
%%time
all_texts, all_labels, all_index = all_data2index(n_fold)

Wall time: 8.98 ms


In [5]:
def index2fold_data(all_texts, all_labels, all_index, fold_num):
    """这里的 texts、labels 是 fold_texts、fold_labels"""
    
    all_fold_data = []   
    
    # 4.根据每折的索引 划分出每折的数据，然后打乱
    # 平均每折的数据量    
    batch_size = int(n_total / fold_num)
    other_texts = []
    other_labels = []
    other_num = 0
    start = 0
    for fold in range(fold_num):
        # 每折的数据量
        num = len(all_index[fold])
        # 从所有数据索引中 索引出 每折数据 对应的text和label的索引
        texts = [all_texts[i] for i in all_index[fold]]
        labels = [all_labels[i] for i in all_index[fold]]
        
        # 如果每折的数据量 > 平均每折的数据量，对该折的数据进行缩减，只取到平均每折的数据量
        if num > batch_size:
            fold_texts = texts[:batch_size]
            fold_labels = labels[:batch_size]           
            other_texts.extend(texts[batch_size:])
            other_labels.extend(labels[batch_size:])
            other_num += num - batch_size
            
        # 如果每折的数据量 < 平均每折的数据量，则将上折剩余的数据补充到该折数据（列表的加法），直到取到平均每折的数据量
        elif num < batch_size:
            end = start + batch_size - num
            # 如果上折剩余的数据量不足以补充该折数据呢，索引就会报错啊？？？？？？？？？？？？？？？
            fold_texts = texts + other_texts[start: end]
            fold_labels = labels + other_labels[start: end]
            # 前面被补充过的数据不再使用
            start = end
        
        # 如果每折的数据量 = 平均每折的数据量，该折的数据进行缩减，只取到平均
        else:
            fold_texts = texts
            fold_labels = labels
        
        # 确保每折的数据量都等同于 平均每折的数据量
        assert batch_size == len(fold_labels)
    
    # 那多出来的数据呢？？？？？？？？？？？？？？？？？？？？？
    
        # 对该折的数据进行打乱，通过列表的索引
        fold_index = list(range(batch_size))
        np.random.shuffle(fold_index)
        shuffle_fold_texts = []
        shuffle_fold_labels = []
        for i in fold_index:
            shuffle_fold_texts.append(fold_texts[i])
            shuffle_fold_labels.append(fold_labels[i])
        
        # 将每折数据添加到 总划分数据里
        data = {'label': shuffle_fold_labels, 'text': shuffle_fold_texts}
        all_fold_data.append(data)
    
    # 记录输出 十折划分后 每折的数据量？？？
    logging.info("Fold lens %s", str([len(fold_data['label']) for fold_data in all_fold_data]))

    return all_fold_data

In [6]:
%%time
all_fold_datas = index2fold_data(all_texts, all_labels, all_index, n_fold)

2020-08-05 18:07:24,063 INFO: Fold lens [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]


Wall time: 5.98 ms


In [7]:
%%time
fold_id = 9

# dev
dev_data = all_fold_datas[fold_id]

# train
train_texts = []
train_labels = []
for i in range(0, fold_id):
    data = all_fold_datas[i]
    train_texts.extend(data['text'])
    train_labels.extend(data['label'])
train_data = {'label': train_labels, 'text': train_texts}

# test
test_data_file = './test_a.csv.zip'
f = pd.read_csv(test_data_file, sep='\t')
texts = f['text'].tolist()
# test数据是没有label，将类别全部标记为 0，是为了方便后面的数据处理 ！！！！！！！！
test_data = {'label': [0] * len(texts), 'text': texts}

Wall time: 2.57 s


#### Build vocab

In [8]:
from collections import Counter

# 新的模块，但是没用到啊 ？？？？？？？？
# from transformers import BasicTokenizer
# basic_tokenizer = BasicTokenizer()

class Vocab():
    
    # 测试：初始化类时，不再运行，模型定义的时候就提前运行了    
    print('测试，类 Vocab 的空白处再定义的时候就运行了')
    
    def __init__(self, train_data):        
        # 测试：定义类时不会运行，初始化类时会运行 ！！！   
        # print('测试，类 Vocab，函数 __init__ 运行了')
        
        # 在所有预料中，最少出现5次才计数
        self.min_count = 5
        # 是为了和哪里的pad对应吗 ？？？？？？？？？？？？
        self.pad = 0
        # reverse操作后，unk的编码就是 1
        self.unk = 1
        
        # 所以word_size都是 4335+2
        self._id2word = ['[PAD]', '[UNK]']
        # 不同于word在类初始化时就填充完毕，extword得调用类时才可以填充
        self._id2extword = ['[PAD]', '[UNK]']
        
        # 14个类别
        self._id2label = []
        self.target_names = []
        
        # 处理传入的参数，可以在这调用后面的函数！！！！！！
        self.build_vocab(train_data)
        
        # 操作是什么意思？？？？？？？？？？？？？？？？
        reverse = lambda x: dict(zip(x, range(len(x))))
        self._word2id = reverse(self._id2word)
        self._label2id = reverse(self._id2label)
        
        
        # 可以调用后面的函数 word_size、label_size，因为没有参数，所以不加括号吗？？？
        logging.info("Build vocab: words %d, labels %d." % (self.word_size, self.label_size))

    def build_vocab(self, data):
        """将每段文本进行分割，统计其中总的出现次数 大于等于5 的单词，加到属性 self._id2word 中"""
                    
        # 测试：初始化类时函数会被调用，代码运行  ！！！！！  
        #print('测试，类 Vocab，函数 build_vocab 运行了')
        
        # 这个新的属性可以在其他地方调用吗？？？？？？？？？？？？？？？？？？？？？
        self.word_counter = Counter()
        
        # 对每个句子进行单词划分，然后计数
        for text in data['text']:
            words = text.split()
            # 不需要if判断？？？？？？？？？？？？？？？？？？？
            for word in words:
                self.word_counter[word] += 1
        
        # 是遍历全部吗？？？？？？？？？？？？？？？？？？？？？？？？？？？
        for word, count in self.word_counter.most_common():
            # 出现次数不到5的word，不添加
            if count >= self.min_count:
                self._id2word.append(word)
        
        # 灵活变化！！！！！！！！！！！！！！
        label2name = {0: '科技', 1: '股票', 2: '体育', 3: '娱乐', 4: '时政', 5: '社会', 6: '教育', 7: '财经',
                      8: '家居', 9: '游戏', 10: '房产', 11: '时尚', 12: '彩票', 13: '星座'}
        
        # 统计每个类别出现的次数，注意类别不是 str，而是 int，所以字典的键是 int
        self.label_counter = Counter(data['label'])
        # label 是 0 到 13 的 index，将每个类别的index（也是原来的int）、对应的中文类别 加到属性 self._id2label、self.target_names里
        for label in range(len(self.label_counter)):
            # 针对每个类别，根据字典的键取值（出现次数count），但是count好像没用到啊 ？？？？？？？？？
            #count = self.label_counter[label]
            self._id2label.append(label)            
            self.target_names.append(label2name[label])


    def load_pretrained_embs(self, embfile):
                    
        # 测试初始化类时，函数不运行，后面调用方法时运行 ！！！    
        #print('测试，类 Vocab，函数 load_pretrained_embs 运行了')
        
        # embfile 是 word2vec.txt文件，第一行是文件的 shape（n_examples, n_features) 
        with open(embfile, encoding='utf-8') as f:
            lines = f.readlines()
            # 默认的分隔符是空格？？？？？？？？？
            items = lines[0].split()
            # n_examples, n_features = 4337, 100
            word_count, embedding_dim = int(items[0]), int(items[1])
        
        # 嵌入层添加数据，文件的第一行是shape！！！！！
        # self._id2extword 的含义是所有的字符（后面的代码还会往里添加） 嵌入层加上它的作用？？？？？？？？？？？
        index = len(self._id2extword)
        embeddings = np.zeros((word_count + index, embedding_dim))
        for line in lines[1:]:
            values = line.split()
            # 第一个数值是哪个字符（字符代码）
            self._id2extword.append(values[0])
            vector = np.array(values[1:], dtype='float64')
            
            # 为什么都要加到嵌入层的第2行上，第一行都是0 ？？？？？？？？？？？？？？？？？？？
            embeddings[self.unk] += vector
            # 刚开始 index=2，所以从第三行开始填充（embeddings层中 index的部分不填充）
            # 不断更改嵌入层里 某层的数值，index不断增加，填充 embeddings层中 word_count的部分
            embeddings[index] = vector
            index += 1
            
        # 第二行加了 word_count 次，把 word2vec文件里的所有向量都加起来了，最后平均下
        embeddings[self.unk] = embeddings[self.unk] / word_count
        # np.std不说明axis的话，是求所有元素的std，但是嵌入层对应 extword 的部分还有很多行（除了第二行）的数据都是0啊？？？？？？？？
        embeddings = embeddings / np.std(embeddings)
        
        # ？？？？？？？？？？？？？？？？？？？？？？？？？
        reverse = lambda x: dict(zip(x, range(len(x))))
        # 这里新创建的属性，被用在了后面的方法里面！！！！！！！！！！！！！！！！！！
        self._extword2id = reverse(self._id2extword)
        
        #print('built_vocab:  self._id2word的个数   self._id2extword的个数')
        # 4337 4337
        #print(len(set(self._id2word)), len(self._id2word))
        # 5978 5978
        #print(len(set(self._id2extword)), len(self._id2extword))    
        logging.info("vocab: word的个数为：%d  %d" % (len(set(self._id2word)), len(self._id2word)) )
        logging.info("vocab: extword的个数为：%d  %d" % (len(set(self._id2extword)), len(self._id2extword)) )
        
        # 生成字典时，相同的键值会合并！！！！！！！！！！！！！！！！
        # 因为重新运行一次，后者的个数会增加，为了不出错，加个确定！！！！！！！！！！！！！！
        assert len(set(self._id2extword)) == len(self._id2extword)

        return embeddings
    
    # 三个转化函数，除了被转化对象 self._word2id  self._extword2id  self._label2id 不一样外，其他都一致
    def word2id(self, xs):
        """xs除了是list，还可能是什么 ？？？？？？？？？"""
        
        # 判断 xs 的类型是否为 list，返回 Bool (参考：https://www.runoob.com/python/python-func-isinstance.html)
        if isinstance(xs, list):
            # 字典的 get 方法, x （键）存在则返回值，不存在则返回 1（参考：https://www.runoob.com/python/att-dictionary-get.html）
            return [self._word2id.get(x, self.unk) for x in xs]
        # 不是 list，单个元素 ？？？？？？？？？？
        return self._word2id.get(xs, self.unk)

    def extword2id(self, xs):        
        if isinstance(xs, list):
            return [self._extword2id.get(x, self.unk) for x in xs]
        return self._extword2id.get(xs, self.unk)

    def label2id(self, xs):        
        if isinstance(xs, list):
            return [self._label2id.get(x, self.unk) for x in xs]
        return self._label2id.get(xs, self.unk)
    
    # property 是 ？？？？？？？？？
    @property
    def word_size(self):
        # 测试：初始化类时，函数运行  ！！！！  
        #print('测试，类 Vocab，函数 word_size 运行了')        
        return len(self._id2word)
    
    @property
    def extword_size(self):        
        return len(self._id2extword)

    @property
    def label_size(self):        
        return len(self._id2label)

测试，类 Vocab 的空白处再定义的时候就运行了


前10000个数据对应 4337-2=4335 个词

In [9]:
%%time
vocab = Vocab(train_data)

2020-08-05 18:07:29,903 INFO: Build vocab: words 4337, labels 14.


Wall time: 3.22 s


#### build word encoder---不同之处

**嵌入层网络部分都一致**，唯一的区别在于最后从嵌入层的词向量中提取特征（文本表示）的方式不同（LSTM vs CNN）

In [10]:
import torch.nn as nn
import torch.nn.functional as F

word2vec_path = './emb/word2vec_all.txt'
dropout = 0.15

# word encoder 中 LSTM 网络参数
word_hidden_size = 128
word_num_layers = 2


class WordLSTMEncoder(nn.Module):
    def __init__(self, vocab):
        super(WordLSTMEncoder, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.word_dims = 100

        self.word_embed = nn.Embedding(vocab.word_size, self.word_dims, padding_idx=0)

        extword_embed = vocab.load_pretrained_embs(word2vec_path)
        extword_size, word_dims = extword_embed.shape
        logging.info("Load extword embed: words %d, dims %d." % (extword_size, word_dims))

        self.extword_embed = nn.Embedding(extword_size, word_dims, padding_idx=0)
        self.extword_embed.weight.data.copy_(torch.from_numpy(extword_embed))
        self.extword_embed.weight.requires_grad = False

        input_size = self.word_dims
        
        # 与 WordCNN 的不同之处，特征提取方式不同 ！！！！！！
        self.word_lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=word_hidden_size,
            num_layers=word_num_layers,
            batch_first=True,
            bidirectional=True
        )

    def forward(self, word_ids, extword_ids, batch_masks):
        # word_ids: sen_num x sent_len
        # extword_ids: sen_num x sent_len
        # batch_masks   sen_num x sent_len

        word_embed = self.word_embed(word_ids)  # sen_num x sent_len x 100
        extword_embed = self.extword_embed(extword_ids)
        batch_embed = word_embed + extword_embed

        if self.training:
            batch_embed = self.dropout(batch_embed)
        
        # 唯一的不同之处 ！！！！！！！！！！！！！！！！！！
        hiddens, _ = self.word_lstm(batch_embed)  # sen_num x sent_len x  hidden*2
        hiddens = hiddens * batch_masks.unsqueeze(2)

        if self.training:
            hiddens = self.dropout(hiddens)
        
        # 输出特征向量
        return hiddens

#### build sent encoder---是一致的

In [11]:
# sent encoder 中 LSTM 网络参数
sent_hidden_size = 256
sent_num_layers = 2


class SentEncoder(nn.Module):
    def __init__(self, sent_rep_size):
        # ？？？？？？？？
        super(SentEncoder, self).__init__()
        
        self.dropout = nn.Dropout(dropout)
        
        # 加入lstm层，论文中也没有啊，lstm层的参数的含义？？？？？？？？？？？？？？
        self.sent_lstm = nn.LSTM(
            input_size=sent_rep_size,     # 输入的特征维度 300
            hidden_size=sent_hidden_size, # 隐藏层的特征维度 256
            num_layers=sent_num_layers,   # lstm隐藏层的层数 2
            batch_first=True,
            bidirectional=True
        )

    def forward(self, sent_reps, sent_masks):
        # sent_reps:  b x doc_len x sent_rep_size （batch_size, max_doc_len，sent_rep_size ) 可以reshape为（sent_num， sent_rep_size ）
        # sent_masks: b x doc_len
        
        # 返回的参数不止一个
        sent_hiddens, _ = self.sent_lstm(sent_reps)  # （b，doc_len，hidden*2）
        sent_hiddens = sent_hiddens * sent_masks.unsqueeze(2)

        if self.training:
            sent_hiddens = self.dropout(sent_hiddens)

        return sent_hiddens

Attention层---也是一致的

In [12]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        # 超类,继承自己，没有从模块里啊 ？？？？？？？？？？？？？
        super(Attention, self).__init__()
        
        # hidden_size 为 Attention 层的隐藏单元数，为sent层中lstm层的隐藏单元数的两倍！！！！！！！！
        
        # 参数的初始化和 tf 不同，没有torch.normal，直接一步到位吗？？？？？？？
        self.weight = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.weight.data.normal_(mean=0.0, std=0.05)
        
        # 没有 torch.zeros ，为何不torch.Tensor(b) ？？？？？
        self.bias = nn.Parameter(torch.Tensor(hidden_size))
        b = np.zeros(hidden_size, dtype=np.float32)
        self.bias.data.copy_(torch.from_numpy(b))
        
        # query ？？？？
        self.query = nn.Parameter(torch.Tensor(hidden_size))
        self.query.data.normal_(mean=0.0, std=0.05)

    def forward(self, batch_hidden, batch_masks):
        # batch_hidden: (b，len，hidden_size=2 * hidden_size of lstm）
        # batch_masks:  （b，len）

        # linear
        key = torch.matmul(batch_hidden, self.weight) + self.bias  # （b，len，hidden） 

        # compute attention
        # query 是一维张量，所以key会减少一个维度
        outputs = torch.matmul(key, self.query)  # （b，len）
        
        # 将outputs中，对应True的位置，用float(-1e32)填充
        # https://blog.csdn.net/jianyingyao7658/article/details/103382654
        masked_outputs = outputs.masked_fill((1 - batch_masks).bool(), float(-1e32))  # （b，len）
        
        # 多分类，softmax 函数激活后再输出，dim=1 是对每一行进行 softmax 激活 ？？？？
        attn_scores = F.softmax(masked_outputs, dim=1)  # （b，len）

        # 对于全零向量，-1e32的结果为 1/len, -inf为nan, 额外补0 ????????????????????????????
        masked_attn_scores = attn_scores.masked_fill((1 - batch_masks).bool(), 0.0)   # （b，len）

        # sum weighted sources ？？？？？？？？？？
        # 批数据之间的矩阵乘法：（b，1, len） 与（b，len，hidden）的 torch.bmm 结果为 （b，1，hidden）
        batch_outputs = torch.bmm(masked_attn_scores.unsqueeze(1), key).squeeze(1)  # （b，1，hidden） 降维为（b，hidden）

        return batch_outputs, attn_scores

#### build model
LSTM 层后面必须要跟一个 attention 层吗 ？？？   
reps 是特征的意思吗 ？？？  
和 WordCNN 一样，最后的全连接层没有 softmax激活，后面训练时就直接送到 CrossEntropy 进行 loss 的计算了 ？？？

In [13]:
class Model(nn.Module):
    def __init__(self, vocab):
        super(Model, self).__init__()
        
        # 不同于 WordCNN，维度有关联
        self.sent_rep_size = word_hidden_size * 2
        
        self.doc_rep_size = sent_hidden_size * 2
        self.all_parameters = {}
        parameters = []
        
        # 不同之处，又多了个 Word Attention 层，难道是因为 LSTM 层后面必须要跟一个 attention 层吗（sent的lstm也跟） ？？？？？？？？
        self.word_encoder = WordLSTMEncoder(vocab)
        parameters.extend(list(filter(lambda p: p.requires_grad, self.word_encoder.parameters())))
        self.word_attention = Attention(self.sent_rep_size)
        parameters.extend(list(filter(lambda p: p.requires_grad, self.word_attention.parameters())))

        self.sent_encoder = SentEncoder(self.sent_rep_size)
        parameters.extend(list(filter(lambda p: p.requires_grad, self.sent_encoder.parameters())))
        self.sent_attention = Attention(self.doc_rep_size)        
        parameters.extend(list(filter(lambda p: p.requires_grad, self.sent_attention.parameters())))

        self.out = nn.Linear(self.doc_rep_size, vocab.label_size, bias=True)
        parameters.extend(list(filter(lambda p: p.requires_grad, self.out.parameters())))

        if use_cuda:
            self.to(device)

        if len(parameters) > 0:
            self.all_parameters["basic_parameters"] = parameters

        logging.info('Build model with LSTM Word Encoder, LSTM Sent Encoder.')

        para_num = sum([np.prod(list(p.size())) for p in self.parameters()])
        logging.info('Model param num: %.2f M.' % (para_num / 1024 / 1024))

    def forward(self, batch_inputs):
        # batch_inputs(batch_inputs1, batch_inputs2): b x doc_len x sent_len
        # batch_masks : b x doc_len x sent_len
        
        batch_inputs1, batch_inputs2, batch_masks = batch_inputs
        batch_size, max_doc_len, max_sent_len = batch_inputs1.shape[0], batch_inputs1.shape[1], batch_inputs1.shape[2]
        batch_inputs1 = batch_inputs1.view(batch_size * max_doc_len, max_sent_len)  # sen_num x sent_len
        batch_inputs2 = batch_inputs2.view(batch_size * max_doc_len, max_sent_len)  # sen_num x sent_len
        batch_masks = batch_masks.view(batch_size * max_doc_len, max_sent_len)  # sen_num x sent_len
        
        # 不同之处，输出的是 hidden，然后要 attention 处理下才能得到 sent_reps（sent层的特征输入）？？？？？
        batch_hiddens = self.word_encoder(batch_inputs1, batch_inputs2, batch_masks)  # sen_num x sent_len x sent_rep_size
        sent_reps, atten_scores = self.word_attention(batch_hiddens, batch_masks)  # sen_num x sent_rep_size
        
        # 后面的都和 WordCNN 一致，不过sent层的输出也成为 sent_hidden（因为也有lstm的原因）？？？？？？
        sent_reps = sent_reps.view(batch_size, max_doc_len, self.sent_rep_size)  # b x doc_len x sent_rep_size
        batch_masks = batch_masks.view(batch_size, max_doc_len, max_sent_len)  # b x doc_len x max_sent_len
        sent_masks = batch_masks.bool().any(2).float()  # b x doc_len

        sent_hiddens = self.sent_encoder(sent_reps, sent_masks)  # b x doc_len x doc_rep_size
        doc_reps, atten_scores = self.sent_attention(sent_hiddens, sent_masks)  # b x doc_rep_size

        batch_outputs = self.out(doc_reps)  # b x num_labels

        return batch_outputs

In [14]:
model = Model(vocab)

2020-08-05 18:07:30,171 INFO: vocab: word的个数为：4337  4337
2020-08-05 18:07:30,172 INFO: vocab: extword的个数为：5978  5978
2020-08-05 18:07:30,173 INFO: Load extword embed: words 5978, dims 100.
2020-08-05 18:07:31,518 INFO: Build model with LSTM Word Encoder, LSTM Sent Encoder.
2020-08-05 18:07:31,519 INFO: Model param num: 4.41 M.


#### optimizer

In [15]:
learning_rate = 2e-4
decay = .75
decay_step = 1000


class Optimizer:
    # 没有super？？？？
    def __init__(self, model_parameters):
        self.all_params = []
        self.optims = []
        self.schedulers = []
        
        # 属性：model.all_parameters
        for name, parameters in model_parameters.items():
            # name专有的属性 startswith 前面model不是有requires_grad的判断了吗 ？？？？？？？
            if name.startswith("basic"):
                # 多个优化器，每个网络对应一个 ？？？？？？？
                optim = torch.optim.Adam(parameters, lr=learning_rate)
                self.optims.append(optim)          
                
                # 参考：https://www.cnblogs.com/zf-blog/p/11262906.html
                l = lambda step: decay ** (step // decay_step)
                # 调整学习率的 lr_scheduler 机制，有两种策略，这里采用 LambdaLR ！！！！！！
                scheduler = torch.optim.lr_scheduler.LambdaLR(optim, lr_lambda=l)
                # 多个 scheduler ？？？？？？？？
                self.schedulers.append(scheduler)
                self.all_params.extend(parameters)
                
            # 这怎么还报错，不是有一个 requires_grad=False
            else:
                Exception("no nameed parameters.")

        self.num = len(self.optims)
    
    # 没有 return和break，不是无限循环？？？？？？？
    def step(self):
        for optim, scheduler in zip(self.optims, self.schedulers):
            # 是训练，然后梯度清零 ？？？？
            # step 和 zero_grad 是类的方法还是 torch 自带的方法？？？？
            optim.step()
            scheduler.step()
            optim.zero_grad()

    def zero_grad(self):
        # 参数逐个（optim）清零
        for optim in self.optims:
            # zero_grad 是 torch 自带的方法吧 ？？？？？？？？
            optim.zero_grad()

    def get_lr(self):
        lrs = tuple(map(lambda x: x.get_lr()[-1], self.schedulers))
        lr = ' %.5f' * self.num
        res = lr % lrs
        return res

#### sentence_split

In [16]:
def sentence_split(text, vocab, max_sent_len=256, max_segment=16):
    """将每个句子分割成长度为 256 的片段，并且总的片段数要小于等于 8，否则进行截断，只取前后 4 个片段"""
    
    # 将每个句子分割成若干个词，生成列表，每个句子的长度可能不一致
    # str.split 默认的分隔符为所有的空字符，包括空格、换行(\n)、制表符(\t)等，这里是 空格
    words = text.strip().split()
    document_len = len(words)
    
    # 间隔为 max_sent_len 的 index [0, 256, 512, 768...，document_len] ！！！！！
    index = list(range(0, document_len, max_sent_len))
    index.append(document_len)
    
    # 将（整个）句子分割成片段，每256个词一个片段，对词频数小于5的进行 unk 编码 ，每个句子最后生成一个嵌套列表
    segments = []
    for i in range(len(index) - 1):
        segment = words[index[i]: index[i + 1]]
        assert len(segment) > 0
        # 此时的 vocab._id2word 包含了所有的单词，不止两个
        # 不存在的词为总的出现次数小于5的（截断），统一标记为 UNK，不会有新的词吧 ？？？？？？？？？？？
        segment = [word if word in vocab._id2word else '<UNK>' for word in segment]
        # 最后一个片段的长度很可能 < 256
        segments.append([len(segment), segment])
    
    # 保证所有句子生成的片段数量都 <= 8
    # 如果每句话生成的片段数量大于 8，则每个句子只取前4个片段（0,1,2,3) 和 后4个片段(-4,-3,-2,-1)，中间的都不要了
    assert len(segments) > 0
    if len(segments) > max_segment:
        segment_ = int(max_segment / 2)
        return segments[:segment_] + segments[-segment_:]
    else:
        return segments


def get_examples(data, vocab, max_sent_len=256, max_segment=8):
    """将训练集中的所有句子分割成长度为 256 的片段，再参照vocab类的 word_ids、 extword_ids 对每个片段进行编码
       返回每个句子的 类别（index形式，而非字符形式）、片段数、每个片段编码后的信息（片段长度、两种编码的结果）"""
    
    # 对属性的新用法：vocab.label2id 不加前面的 _ ！！！！！！！！！！！
    # label2id 是个字典方法，是vocab类中前面定义的属性，而不是后面定义的方法（函数） ！！！！！
    label2id = vocab.label2id
    examples = []
    
    # 对于每一个句子 和其对应的 类别（int）
    for text, label in zip(data['text'], data['label']):
        # label 为单个字符，这里是根据键（字符），索引值（类别的id，index）
        id = label2id(label)

        # segment words（用于sent层？？？？？？？？？？？？）
        sents_words = sentence_split(text, vocab, max_sent_len, max_segment)
        doc = []
        
        # 列表的新方法，可以直接对嵌套列表的内层列表进行迭代 ！！！！！！！！！！
        # 针对每个句子 sents_words 的每个片段 sent_words，将片段中的每个词进行编码，生成列表！！！
        # 分别参照vocab._word2id、vocab._extword2id，   字 --> index
        for sent_len, sent_words in sents_words:
            word_ids = vocab.word2id(sent_words)
            # 没有新的extwords，只是参考的对象（额外训练的）不同
            extword_ids = vocab.extword2id(sent_words)
            # word_ids 和 extword_ids 都是新生成的列表
            doc.append([sent_len, word_ids, extword_ids])
            
        # 每个 doc 对应一个句子
        examples.append([id, len(doc), doc])

    logging.info('Total %d docs.' % len(examples))
    return examples

#### data_iter

In [17]:
def batch_slice(data, batch_size):
    """data是什么？？？ list？？？"""
    # np.neil 向上取整，保证批数量能刚好把所有样本训练完
    batch_num = int(np.ceil(len(data) / float(batch_size)))
    
    # 对每个批次的数据，如果不是最后 1 个批次（i < batch_num - 1），则该批次数据量为规定的 batch_size
    # 如果是最后一批次数据，则将剩下的数据作为该批次的数据量
    for i in range(batch_num):
        cur_batch_size = batch_size if i < batch_num - 1 else len(data) - batch_size * i
        # 从全部数据中，分别取出每批次数据，对data一个一个索引（相当于切片），生成嵌套列表
        docs = [data[i * batch_size + b] for b in range(cur_batch_size)]

        yield docs


def data_iter(data, batch_size, shuffle=True, noise=1.0):

    # 随机打乱数据，这里的随机打乱有什么意义，本来就是按照每个句子的片段长度（句子的长度）进行排序的
    if shuffle:
        np.random.shuffle(data)
    batched_data = []
    
    # example就是上个代码段里 examples 的单个元素，example[1] 就是每个句子的片段数 len（doc）
    lengths = [example[1] for example in data]
    # 为什么要给长度加个 -1和1 之间的随机数啊。每个对应的随机数都不同，是为了方便给长度相同的片段也进行排序吗 ？？？？？？？
    # -1和1 之间的数，不会改变3，4，5之间的排位，只会对 4 4 4 之间的排位有影响
    noisy_lengths = [- (l + np.random.uniform(- noise, noise)) for l in lengths]
    # 生成升序排序后的 index，sorted_indices的第一个index，对应长度最短的句子
    sorted_indices = np.argsort(noisy_lengths).tolist()
    # 按照片段数的从少到多，索引对应的句子
    sorted_data = [data[i] for i in sorted_indices]
    
    # extend（list(a)) 不就等同于 append（a）吗，为什么不直接append，是因为 yield的原因吗？？？？？
    # 每次extend的都是一个 batch，是一个嵌套列表
    batched_data.extend(list(batch_slice(sorted_data, batch_size)))
    
    # 这里又进行打乱，
    if shuffle:
        np.random.shuffle(batched_data)
    
    # batch 是个嵌套列表，每个元素是一个句子
    for batch in batched_data:
        yield batch 

#### scores

In [18]:
from sklearn.metrics import f1_score, precision_score, recall_score


def get_score(y_ture, y_pred):
    y_ture = np.array(y_ture)
    y_pred = np.array(y_pred)
    f1 = f1_score(y_ture, y_pred, average='macro') 
    p = precision_score(y_ture, y_pred, average='macro') 
    r = recall_score(y_ture, y_pred, average='macro') 
    
    # 和下面的函数对照，精简下函数
    #return str((reformat(p, 2), reformat(r, 2), reformat(f1, 2))), reformat(f1, 2)
    return reformat(p), reformat(r), reformat(f1)


def reformat(num):
    """保留4位小数的格式化输出"""
    #return float(format(num, '0.' + str(n) + 'f'))
    return float(format(num, '0.4f'))

#### 模型训练
相比于bert.ipnb，修改处为：
- init() 里加一个 self.test_data，对测试集数据的分片段处理
- 修改了模型的保存地址
- 修改了train和test的 batch_size 
- 修改了_eval函数，为了适应测试模式

test数据集填充了类别0，因为 get_examples()函数、batch2Tensor()函数、都涉及到 label 的处理

In [19]:
import time
from sklearn.metrics import classification_report

# ？？？？？？
clip = 5.0
# 结束训练的标志
early_stops = 3
# 每隔 50 个批次打印一次训练信息
log_interval_print = 50

epochs = 2

# 修改之处
test_batch_size = 16
train_batch_size = 16
save_model_path = './rnn.bin'
save_test_path = './rnn.csv'

class Trainer():
    def __init__(self, model, vocab):
        self.model = model
        self.report = True
        
        # 训练集的切割（片段）与编码，此时的训练集已经是处理过后的，但是每个样本（句子）的长度不一致（片段数不同）
        # get_examples返回的训练数据是列表形式，所以这里可以控制训练集的大小 ！！！！！！！！
        # 控制到 4个批次，方便后面查看结果
        self.train_data = get_examples(train_data, vocab)
        #self.train_data = get_examples(train_data, vocab)[:1000]
        self.batch_num = int(np.ceil(len(self.train_data) / float(train_batch_size)))
        # 验证集的切割（片段）与编码
        self.dev_data = get_examples(dev_data, vocab)
        self.test_data = get_examples(test_data, vocab)

        # criterion 损失函数
        self.criterion = nn.CrossEntropyLoss()
         # optimizer 和一般优化器的区别，传入了参数 ？？？
        self.optimizer = Optimizer(model.all_parameters)
        
        # label name 中文字符
        self.target_names = vocab.target_names     

        # count
        self.step = 0
        self.early_stop = -1
        self.best_train_f1, self.best_dev_f1 = 0, 0
        #  last_epoch  ？？？
        self.last_epoch = epochs

    def train(self):
        logging.info('Start training...')
        # 这里只迭代了一轮，epochs=1
        for epoch in range(1, epochs + 1):
            # 调用后面的方法，计算训练集的 f1
            train_f1 = self._train(epoch)
            # 调用后面的方法，计算验证集的 f1，不传入test，是验证模式
            dev_f1 = self._eval(epoch)
            
            # 本轮训练有进步，就保存训练结果，继续训练（self.early_stop = 0）
            if self.best_dev_f1 <= dev_f1:
                logging.info(
                    "Exceed history dev f1 = %.2f, current dev f1 = %.2f" % (self.best_dev_f1, dev_f1))
                logging.info('\n')
                
                torch.save(self.model.state_dict(), save_model_path)
                
                self.best_train_f1 = train_f1
                self.best_dev_f1 = dev_f1
                
                # 本轮效果有进步，还可以继续训练，不需要停止 ？？？？？？？？？？？
                self.early_stop = 0
                
            # 如果本轮训练结果不行（self.best_dev_f1 > dev_f1），并且已经连续（必须连续）三轮训练结果都没有超过三轮前的 self.best_dev_f1
            # 则在第 epoch - early_ stops 轮就应该停止训练了（这个信息哪里还用到，再次训练吗 ？？？？？）
            else:
                # 因为这轮的训练效果已经开始变差了，所以要提前一轮就结束训练 ？？？？？
                self.early_stop += 1
                if self.early_stop == early_stops:
                    logging.info( "Eearly stop in epoch %d, best train f1: %.2f, dev f1: %.2f" % 
                                 (epoch - early_stops, self.best_train_f1, self.best_dev_f1)
                                )
                    # 作用 ？？？？？？？？？？？？？？？
                    self.last_epoch = epoch
                    break

    def test(self):
        """不返回参数啊，也不打印结果，但是会预测保存的结果 ！！！！！！"""
        self.model.load_state_dict(torch.load(save_model_path))        
        self._eval(self.last_epoch + 1, test=True)
    
    def _train(self, epoch):
        # 每轮训练都要梯度清零
        self.optimizer.zero_grad()
        # torch 自带的，进入训练状态（是否必须？ 是还没有传入训练样本） ？？？？？？？？？？？？？
        self.model.train()
        
        # 直接 log 输出不就行了
        start_time = time.time()
        epoch_start_time = time.time()
        
        overall_losses = 0
        # 每轮训练中，所有批次的损失之和
        losses = 0
        batch_idx = 1
        # 所有训练样本的类别（一轮）
        y_pred = []
        y_true = []
        # 批量输出训练集
        for batch_data in data_iter(self.train_data, train_batch_size, shuffle=True):
            # pytorch会自动释放不用的显存gpu，这句话的意义是为了在 Nvidia-smi 命令里也能看到显存被释放
            # https://blog.csdn.net/qq_29007291/article/details/90451890
            torch.cuda.empty_cache()
            
            # 实现训练集的特征构建（额外的零填充）和张量转化
            # batch_labels：（batch_size=128, )
            batch_inputs, batch_labels = self.batch2tensor(batch_data)
            
            # 前向传播计算输出，再计算损失
            # batch_outputs：（batch_size, num_labels) 
            batch_outputs = self.model(batch_inputs) 
            # 但是model最后的全连接层没有 softmax激活，输出的是预测概率吗，怎么直接就进行 loss 的计算了 ？？？？？？？？？
            loss = self.criterion(batch_outputs, batch_labels)
            
            # 根据损失自动进行反向传播（修改参数？？？？）
            loss.backward()
            
            # 损失值的计算不放在反向传播的前面吗 ？？？？？？？？？？
            # 损失值是单个 mse 还是 se ？？？？？？？？？？？？？？
            # 得到损失值，必须要换为 cpu 格式吗
            # detach     https://blog.csdn.net/Z_lbj/article/details/79604104
            loss_value = loss.detach().cpu().item()
            # losses 统计 50 批次的总损失
            losses += loss_value
            # overall_losses 统计每轮的总损失
            overall_losses += loss_value
            
            # 没有softmax激活，是预测概率转化为预测类别吗 ？？？？
            # torch.max(batch_outputs, dim=1)[1] 返回位置索引，也是类别标记（int） ！！！！！！！！！
            y_pred.extend(torch.max(batch_outputs, dim=1)[1].cpu().numpy().tolist())
            y_true.extend(batch_labels.cpu().numpy().tolist())
            
            # 
            nn.utils.clip_grad_norm_(self.optimizer.all_params, max_norm=clip)
            
            # 每批次训练
            # optimizer.step()  scheduler.step()     https://blog.csdn.net/qq_20622615/article/details/83150963
            for optimizer, scheduler in zip(self.optimizer.optims, self.optimizer.schedulers):
                # 调用step方法，是不是只需调用一次就行了，这个重复了吧 ？？？？？？
                # 只有用了optimizer.step()，模型才会更新
                optimizer.step()
                # scheduler.step()是对学习率 lr 进行调整
                scheduler.step()
            # 批次训练时，梯度清零的原因和作用： 防止 上次计算的梯度 和 本次计算的梯度 进行累加
            # https://blog.csdn.net/u011959041/article/details/102760868
            # https://blog.csdn.net/scut_salmon/article/details/82414730
            self.optimizer.zero_grad()
            
            # 每批次都加 1,并且每轮训练完都不清零 ！！！！！！！！！！！
            self.step += 1
            
            # 每隔 log_interval = 50 个批次，打印一次训练信息
            if batch_idx % log_interval_print == 0:
                # 统计训练 50 个批次所需的时间
                elapsed = time.time() - start_time
                # 当前的学习率
                lrs = self.optimizer.get_lr()
                logging.info(
                    '| epoch {:3d} | step {:3d} | batch {:3d}/{:3d} | lr{} | loss {:.4f} | batch time {:.2f}s/batch'.format(
                        epoch, self.step, batch_idx, self.batch_num, lrs,
                        # 统计每批次的平均损失（50批计算一次）
                        losses / log_interval_print,
                        # 统计训练 50 个批次时，平均每个批次所需的时间
                        elapsed / log_interval_print))
                # 清零，统计下一个50 批次的平均每批次损失
                losses = 0
                start_time = time.time()
            # 每训练一批次，加一
            batch_idx += 1
            
        # 每轮一次
        # 平均每批次的总损失 = 每轮的总损失 /  总批次数 （每轮计算一次）
        overall_losses /= self.batch_num
        # reformat，精确 4 位小数
        overall_losses = reformat(overall_losses)
        # 计算训练一轮所需的时间
        during_time = time.time() - epoch_start_time    
        # 评价整轮样本的训练结果
        p, r, f1 = get_score(y_true, y_pred)     
        
        # 每轮打印一次信息
        logging.info(
            '| epoch {:3d} | Precision {} | Recall {} | F1 {} | loss {:.4f} | time {:.2f}'.format(epoch, p, r, f1, 
                                                                                                  overall_losses, during_time))
        
        # 如果预测的信息包含所有类别，就可以输出计算的信息了
        if set(y_true) == set(y_pred) and self.report:
            # digits=4，保留四位小数
            report = classification_report(y_true, y_pred, digits=4, target_names=self.target_names)
            logging.info('\n' + report)

        return f1

    def _eval(self, epoch, test=False):
        # epoch 有意义吗？ 每次预测的结果会有出入吗（dropout会影响吗？？？）
        
        """ 测试模式和验证模式合到一起： test参数
           pytorch中model eval和torch no grad()的区别:    
           https://blog.csdn.net/songyu0120/article/details/103884586
        """
        # torch的自带方法 eval，从训练模式切换到验证模式        
        self.model.eval()
        
        start_time = time.time()
        y_pred = []
        y_true = []
        
        # 更进一步加速和节省gpu空间（不用计算和存储 gradient）
        # torch.no_grad() 是一个上下文管理器，被该语句 wrap 起来的部分将不会track 梯度
        # https://blog.csdn.net/weixin_46559271/article/details/105658654
        with torch.no_grad():
            # 新加的
            if test:
                data = self.test_data
            else:
                data = self.dev_data
                
            for batch_data in data_iter(data, test_batch_size, shuffle=False):
                torch.cuda.empty_cache()
                batch_inputs, batch_labels = self.batch2tensor(batch_data)
                # 模型的架构和参数还都存在
                batch_outputs = self.model(batch_inputs)
                y_pred.extend(torch.max(batch_outputs, dim=1)[1].cpu().numpy().tolist())
                y_true.extend(batch_labels.cpu().numpy().tolist())
            
            during_time = time.time() - start_time
            
            if test:
                df = pd.DataFrame({'label': y_pred})
                # index 参数在提交结果时是否重要 ？？？？？？？
                df.to_csv(save_test_path, index=False, sep=',')
                logging.info('Test data predicted over')
                return ;
                
            # 对于验证集数据，打印一下评估信息    
            else:
                # 每轮评估一下验证性能（验证集和测试集数据都有 y_true，但是测试集的没有意义）
                p, r, f1 = get_score(y_true, y_pred)
            
                logging.info(
                    '| dev | Precision {} | Recall {} | F1 {} | time {:.2f}s'.format(epoch, p, r, f1, during_time))
                
                if set(y_true) == set(y_pred) and self.report:
                    report = classification_report(y_true, y_pred, digits=4, target_names=self.target_names)
                    logging.info('\n' + report)
        # 验证模式返回 F1
        return f1

    def batch2tensor(self, batch_data):
        """对批量数据的处理: 通过生成 batch_size（128）个 8 * 256 的零张量，实现特征值的输入和长度填补（补零）
            batch_data 同 examples 的结构一致，只是包含的样本数不同
            batch_data = [
            [label, doc_len, [[sent_len, [sent_id0, ...], [sent_id1, ...]], ..., [sent_idn1, ...]]]
            ]
        """
        batch_size = len(batch_data)
        
        # doc就是一个sentence
        # 统计每个句子的类别
        doc_labels = []
        # 统计每个句子分割成的片段数
        doc_lens = []
        # 统计每个句子的最长的片段长度（很可能都是256）
        doc_max_sent_len = []
        
        # 对于每个句子
        for doc_data in batch_data:
            # 句子的类别
            doc_labels.append(doc_data[0])
            # 句子分割的片段总数
            doc_lens.append(doc_data[1])
            
            # sent就是句子的片段（segment）
            # 统计每个片段的长度（字符数）
            sent_lens = [sent_data[0] for sent_data in doc_data[2]]
            # 一般来说，最长的应该是 256，但是可能有的整个句子的长度都小于 256，所以这个不一定统一 ！！！！！！！！！
            max_sent_len = max(sent_lens)
            doc_max_sent_len.append(max_sent_len)
        
        # 统计所有句子中，最多的片段数（8） 以及 最长的片段长度（256），后面进行填充 ？？？？？？？
        max_doc_len = max(doc_lens)
        max_sent_len = max(doc_max_sent_len)
        
        # 统一按照最大的长度进行填充（max_doc_len, max_sent_len），不存在的地方用 0 填充
        batch_inputs1 = torch.zeros((batch_size, max_doc_len, max_sent_len), dtype=torch.int64)
        batch_inputs2 = torch.zeros((batch_size, max_doc_len, max_sent_len), dtype=torch.int64)
        # 有字符的地方统一用 1 标记 ！！！！！！！！！！！！！
        batch_masks = torch.zeros((batch_size, max_doc_len, max_sent_len), dtype=torch.float32)
        # LongTensor
        batch_labels = torch.LongTensor(doc_labels)
        
        # 对于每个句子（index）
        for b in range(batch_size):
            # 对于每个句子的每个片段（index）
            for sent_idx in range(doc_lens[b]):
                # 对于每个片段（data）： sent_data = [sent_len, word_ids, extword_ids]
                sent_data = batch_data[b][2][sent_idx]
                # 对于片段里的每个字符（index形式）：sent_data[0] = sent_len
                for word_idx in range(sent_data[0]):
                    # 填充字符对应的 word_ids、extword_ids 编码
                    batch_inputs1[b, sent_idx, word_idx] = sent_data[1][word_idx]
                    batch_inputs2[b, sent_idx, word_idx] = sent_data[2][word_idx]
                    
                    # 有字符的地方统一用 1 标记
                    batch_masks[b, sent_idx, word_idx] = 1
        
        # 将数据存在显存上
        if use_cuda:
            batch_inputs1 = batch_inputs1.to(device)
            batch_inputs2 = batch_inputs2.to(device)
            batch_masks = batch_masks.to(device)
            batch_labels = batch_labels.to(device)

        return (batch_inputs1, batch_inputs2, batch_masks), batch_labels

本来是0.55s/batch

每轮的训练速度和 batch_size 大小不是线性关系，batch_size 越大，相对来说训练速度越快，但是费显存

In [20]:
%%time
trainer = Trainer(model, vocab)
trainer.train()

2020-08-05 18:08:20,501 INFO: Total 9000 docs.
2020-08-05 18:08:25,471 INFO: Total 1000 docs.
2020-08-05 18:12:47,283 INFO: Total 50000 docs.
2020-08-05 18:12:47,284 INFO: Start training...
2020-08-05 18:13:53,730 INFO: | epoch   1 | step  50 | batch  50/563 | lr 0.00020 | loss 2.4239 | batch time 1.33s/batch
2020-08-05 18:14:58,130 INFO: | epoch   1 | step 100 | batch 100/563 | lr 0.00020 | loss 2.1542 | batch time 1.29s/batch
2020-08-05 18:16:00,336 INFO: | epoch   1 | step 150 | batch 150/563 | lr 0.00020 | loss 1.7478 | batch time 1.24s/batch
2020-08-05 18:17:00,739 INFO: | epoch   1 | step 200 | batch 200/563 | lr 0.00020 | loss 1.5995 | batch time 1.21s/batch
2020-08-05 18:18:05,233 INFO: | epoch   1 | step 250 | batch 250/563 | lr 0.00020 | loss 1.4514 | batch time 1.29s/batch
2020-08-05 18:19:06,673 INFO: | epoch   1 | step 300 | batch 300/563 | lr 0.00020 | loss 1.3949 | batch time 1.23s/batch
2020-08-05 18:20:11,495 INFO: | epoch   1 | step 350 | batch 350/563 | lr 0.00020 | 

Wall time: 30min 19s


In [1]:
9000/16

562.5

In [ ]:
trainer.test()

In [ ]:
https://blog.csdn.net/weixin_38673554/article/details/103022918
    
https://blog.csdn.net/yepeng_xinxian/article/details/95519152
    
https://blog.csdn.net/qq_39938666/article/details/86611474
    
https://www.cnblogs.com/wanghui-garcia/p/11514502.html